# IBM Data Science Capstone
# Week 3 Assignment
# Canadian Postcodes

Import Relevant Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup


Scrape Wikipedia Postcodes page intp pandas dataframe

In [85]:
#url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050."   #old URL
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

df = (pd.read_html(str(soup.table))[0])

#drop the first row and add column headers
df.drop(axis=0, index=0, inplace=True)
headers = ['Postcode', 'Borough', 'Neighborhood'] 
df.columns = headers

#drop rows with unassigned Boroughs
df = df[df['Borough'] != "Not assigned"].reset_index(drop=True) 
df



,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


Remove rows with unassigned Neighborhoods

In [86]:
na = df[df['Neighborhood']=="Not assigned"]
for index in range(len(na)):
    r = na.iloc[index]
    r['Neighborhood']=r['Borough']
    df.iloc[na.index[0]]['Neighborhood'] = r['Borough']

df.head(8)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,Rouge


In [87]:
aggregations = {
    'Neighborhood':lambda x:', '.join(x)}
g = df.groupby('Postcode').agg(aggregations).reset_index()
x = df.drop('Neighborhood',axis=1)
m = pd.merge(x, g, on='Postcode', how='left')
dd = m.drop_duplicates()
dd.to_csv("Toronto_Q1_Answer.csv", index=False)
dd # dd means dropped duplicates

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park
...,...,...,...
192,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
195,M4Y,Downtown Toronto,Church and Wellesley
196,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
197,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway..."
